# 1. Download Data



In [ ]:
!pip install sklearn-crfsuite

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 34.4 MB/s eta 0:00:00


# Preprocessing: ตัดคำและติด POS Tag ก่อน

In [ ]:
!pip install pythainlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 81.4 MB/s eta 0:00:00


#feature เป็นแค่คำอย่างเดียว

In [ ]:
from lst20utils import extract_sentences_from_files, extract_entities, retag

#ดึง tokens จาก train_auto_tok.tsv

ลองดึง label กับ sentence แบบมี white space

In [ ]:
token_seq_list = []
sentence = []
label_seq = []
label_seq_list = []
with open('/content/train_auto_tok.tsv', 'r') as f:
    original_list = f.readlines()
    for word_and_tag in original_list:
        word = word_and_tag.split('\t')[0]
        if word_and_tag == '\n':
            token_seq_list.append(sentence)
            sentence = []
            label_seq_list.append(label_seq)
            label_seq = []
        else:
            sentence.append(word)
            tag = word_and_tag.split('\t')[-1].strip()
            label_seq.append(tag)
    # Append any remaining sequence
    if sentence:
        token_seq_list.append(sentence)

print(token_seq_list[-10:])
print(label_seq_list[-10:])

[['ทาง', 'ด้าน', ' ', 'พ.ต.', 'อ', ' ', 'ส', 'รา', 'ยุทธ', ' ', 'สงวน', 'โภ', 'คัย', ' ', 'ผู้กำกับ', ' ', 'สภ.', 'เมือง', 'พัทยา', ' ', 'ซึ่ง', 'ได้', 'เดินทาง', 'มา', 'รับ', 'ตำแหน่ง', 'ใหม่', 'หลัง', 'ถูก', 'สั่ง', 'ย้าย', 'ราชการ', 'ไป', 'อยู่', 'จังหวัด', 'ชลบุรี', 'ชั่วคราว'], ['และ', 'ได้', 'กล่าวว่า', ' ', 'ใน', 'วัน', 'พ', 'รุ้ง', 'นี้', 'จะ', 'นำ', 'ตัวผู้', 'ต้อง', 'มา', 'ไป', 'ที่เกิดเหตุ', 'เพื่อ', 'ทำ', 'แผน', 'ประกอบ', 'คำรับสารภาพ', 'เพื่อ', 'เป็น', 'แนวทาง', 'ทาง', 'คดี', 'ต่อไป', 'ดังกล่าว'], ['ทำเนียบฯ', 'เงียบเหงา', 'คณะ', 'ปฏิรูป', 'ฯ', 'ให้', 'สื่อ', 'ทำหน้าที่', 'ปกติ'], ['ทำเนียบรัฐบาล', ' ', 'บรรยากาศ', 'เงียบเหงา', ' ', 'คณะ', 'ปฏิรูป', 'ฯ', 'ให้', 'สื่อมวลชน', 'ทำหน้าที่', 'ตามปกติ'], ['(', '21', ' ', 'ก.ย.', ')', ' ', 'ผู้สื่อข่าว', 'รายงาน', 'จาก', 'ทำเนียบรัฐบาล', 'ว่า', ' ', 'หลัง', 'คณะ', 'ปฏิรูป', 'การปกครอง', 'ใน', 'ระบอบประชาธิปไตย', 'อัน', 'มี', 'พระมหากษัตริย์', 'ทรง', 'เป็น', 'ประมุข', ' ', 'มี', 'คำสั่ง', 'ให้', 'วันนี้', ' ', '(', '21', ' ', 'ก.ย

In [ ]:
token_seq_list2 = []
sentence2 = []
label_seq2 = []
label_seq_list2 = []
with open('/content/dev_auto_tok.tsv', 'r') as f:
    original_list = f.readlines()
    for word_and_tag in original_list:
        word = word_and_tag.split('\t')[0]
        if word_and_tag == '\n':
            token_seq_list2.append(sentence2)
            sentence2 = []
            label_seq_list2.append(label_seq2)
            label_seq2 = []
        else:
            sentence2.append(word)
            tag = word_and_tag.split('\t')[-1].strip()
            label_seq2.append(tag)
    # Append any remaining sequence
    if sentence2:
        token_seq_list2.append(sentence2)

print(token_seq_list2[:10])
print(label_seq_list2[:10])

[['"', 'อ.', 'สม', 'พจน์', '"', 'แต่ง', '"', 'แพนด้า', 'น่ารัก', '"', ' ', 'ตุน', 'เพลง', 'รอ', '"', 'ลูกกรุง', '"', 'กลับมา', 'อีก'], ['หลังจากที่', 'มี', 'ข่าว', 'ว่า', 'ได้', 'แต่งเพลง', 'แพนด้า', 'น่ารัก', ' ', 'เพื่อ', 'มอบให้', 'ลูก', 'ของ', 'ช่วง', 'ช่วง', 'และ', 'หลินฮุ่ย', ' ', 'แม้', 'จะ', 'ยัง', 'ไม่', 'ทราบ', 'ว่า', 'ลูก', 'หมี', 'แพนด้า', 'น้อย', 'ชื่อ', 'อะไรก็ตาม', ' ', 'อ.', 'สม', 'พจน์', ' ', 'สิงห์', 'สุวรรณ', ' ', 'ได้', 'ให้', 'สัมภาษณ์', 'ที่', 'บ้าน', 'ฝัน', 'ตะวัน', 'รัญจวน', ' ', 'ลาดพร้าว', ' ', '124', ' ', 'ว่า'], ['"', 'สาเหตุ', 'ที่', 'ผม', 'แต่งเพลง', 'แพนด้า', 'น่ารัก', ' ', 'ก็', 'เพราะ', 'อยาก', 'จะ', 'มีส่วน', 'ร่วมใน', 'การ', 'ต้อนรับ', 'ครอบครัว', 'หมี', 'แพนด้า', ' ', 'ซึ่ง', 'ทางการ', 'จีน', 'ส่ง', 'มา', 'อยู่', 'ที่', 'ประเทศ', 'ไทย', 'ทูต', 'วัฒนธรรม', ' ', 'ผม', 'แต่งเพลง', 'นี้', 'เสร็จ', 'เรียบร้อย', 'แล้ว', ' ', 'ถึง', 'ได้', 'พบ', 'กับ', 'น้อง', 'ออมสิน', 'ที่', 'บ้าน', 'ฝัน', 'ฯ', ' ', 'เมื่อ', 'ทาบทาม', 'ให้', 'ร้องเพลง', 'นี้', 'น้อง', 'ออ

In [ ]:
token_seq_list.extend(token_seq_list2)
label_seq_list.extend(label_seq_list2)

In [ ]:
len(token_seq_list)

68930

#tokenize text จาก .txt

In [ ]:
with open('dev_set.txt', 'r', encoding='utf-8') as f:
    # Read the contents of the file into a string variable
    text = f.read()

# Split the text into sentences using the newline character as a delimiter
sentences = text.split('\n')

# Split each sentence into words using the '|' character as a delimiter
dev_seq_list = [sentence.split('|') for sentence in sentences]

# Print the resulting list of word lists
print(len(dev_seq_list))

5620


In [ ]:
with open('test_set.txt', 'r', encoding='utf-8') as f:
    # Read the contents of the file into a string variable
    text = f.read()

# Split the text into sentences using the newline character as a delimiter
sentences = text.split('\n')

# Split each sentence into words using the '|' character as a delimiter
test_seq_list = [sentence.split('|') for sentence in sentences]

# Print the resulting list of word lists
print(len(test_seq_list))

5250


#Featurize

In [ ]:
def prev(i, tokens):
    if i-1 > 0 :
      prev = tokens[i-1][0]
    else:
      prev = 'START-WORD'
    return {'prev': prev}

In [ ]:
def current(i, tokens):
    return {'current': tokens[i][0]}

In [ ]:
def next(i, tokens):
    if i+1 < len(tokens) :
      next = tokens[i+1][0]
    else:
      next = 'END-WORD'
    return {'next': next}

In [ ]:
def conjunctive_word(i, tokens):
    words =[]
    posi = [-2, -1, 0, 1, 2]
    position_str = ' '.join([str(x) for x in posi])
    for position in posi:
        if 0 <= i + position < len(tokens):
            words.append(tokens[i+position][0])
    return {'conj-word': '-'.join(words)}

In [ ]:
def prev_pos(i, tokens):
    if i-1 > 0 :
      prev = tokens[i-1][1]
    else:
      prev = 'START-WORD'
    return {'prev_pos': prev}

In [ ]:
def current_pos(i, tokens):
    return {'current_pos': tokens[i][1]}

In [ ]:
def next_pos(i, tokens):
    if i+1 < len(tokens) :
      next = tokens[i+1][1]
    else:
      next = 'END-WORD'
    return {'next_pos': next}

In [ ]:
def conjunctive_pos(i, tokens):
    words =[]
    posi = [-1, 0, 1]
    position_str = ' '.join([str(x) for x in posi])
    for position in posi:
        if 0 <= i + position < len(tokens):
            words.append(tokens[i+position][1])
    return {'conj-pos': '-'.join(words)}

In [ ]:
s = ['ทาง', 'ด้าน', ' ', 'พ.ต.', 'อ', ' ', 'ส', 'รา', 'ยุทธ', ' ', 'สงวน', 'โภ', 'คัย', ' ', 'ผู้กำกับ', ' ', 'สภ.', 'เมือง', 'พัทยา', ' ', 'ซึ่ง', 'ได้', 'เดินทาง', 'มา', 'รับ', 'ตำแหน่ง', 'ใหม่', 'หลัง', 'ถูก', 'สั่ง', 'ย้าย', 'ราชการ', 'ไป', 'อยู่', 'จังหวัด', 'ชลบุรี', 'ชั่วคราว']

In [ ]:
from pythainlp import pos_tag
s_tag = pos_tag(s, corpus='orchid_ud')

In [ ]:
print(s_tag)

[('ทาง', 'ADP'), ('ด้าน', 'NOUN'), (' ', 'PUNCT'), ('พ.ต.', 'NOUN'), ('อ', 'NOUN'), (' ', 'PUNCT'), ('ส', 'NOUN'), ('รา', 'NOUN'), ('ยุทธ', 'NOUN'), (' ', 'PUNCT'), ('สงวน', 'NOUN'), ('โภ', 'NOUN'), ('คัย', 'NOUN'), (' ', 'PUNCT'), ('ผู้กำกับ', 'ADP'), (' ', 'PUNCT'), ('สภ.', 'NOUN'), ('เมือง', 'NOUN'), ('พัทยา', 'NOUN'), (' ', 'PUNCT'), ('ซึ่ง', 'SCONJ'), ('ได้', 'AUX'), ('เดินทาง', 'VERB'), ('มา', 'AUX'), ('รับ', 'VERB'), ('ตำแหน่ง', 'NOUN'), ('ใหม่', 'ADJ'), ('หลัง', 'NOUN'), ('ถูก', 'AUX'), ('สั่ง', 'VERB'), ('ย้าย', 'VERB'), ('ราชการ', 'NOUN'), ('ไป', 'AUX'), ('อยู่', 'AUX'), ('จังหวัด', 'NOUN'), ('ชลบุรี', 'NOUN'), ('ชั่วคราว', 'ADJ')]


In [ ]:
next_pos(3, s_tag)

{'next_pos': 'NOUN'}

In [ ]:
conjunctive_word(10, s_tag)

{'conj-word': 'ยุทธ- -สงวน-โภ-คัย'}

In [ ]:
#tok_seq_list = [[(w if w != '' else ' ') for w in sentence] for sentence in token_seq_list]
#tok_seq_list = [[(x[0], x[1]) for x in pos_tag(s, corpus='pud')] for s in tok_seq_list]

In [ ]:
from pythainlp import pos_tag
tok_seq_list = [[(x[0], x[1]) for x in pos_tag(s, corpus='orchid_ud')] for s in token_seq_list]
def featurize(feature_function_list, tokens):
    feature_dict_seq = []
    for i in range(len(tokens)):
        feature_dict = {}
        for feature_fn in feature_function_list:
            if feature_fn == 'conjunctive_word':
                if i == 0 or i == len(tokens)-1:
                    pass
                else:
                    feature_dict.update(feature_fn(i, tokens))
            else:
                feature_dict.update(feature_fn(i, tokens))
        feature_dict_seq.append(feature_dict)
    if len(tokens) > 2:
        feature_dict_seq[0].pop('conj-word', None)
        feature_dict_seq[-1].pop('conj-word', None)
        feature_dict_seq[0].pop('conj-pos', None)
        feature_dict_seq[-1].pop('conj-pos', None)
    return feature_dict_seq
feature_functions = [current, current_pos, next, next_pos, prev, prev_pos, conjunctive_word, conjunctive_pos]
feature_seq_list = [featurize(feature_functions, token_list) for token_list in tok_seq_list]
feature_seq_list[-1]

[{'current': 'อีก',
  'current_pos': 'DET',
  'next': 'ทั้งเมือง',
  'next_pos': 'DET',
  'prev': 'START-WORD',
  'prev_pos': 'START-WORD'},
 {'current': 'ทั้งเมือง',
  'current_pos': 'DET',
  'next': 'วอน',
  'next_pos': 'NOUN',
  'prev': 'START-WORD',
  'prev_pos': 'START-WORD',
  'conj-word': 'อีก-ทั้งเมือง-วอน-ซาน',
  'conj-pos': 'DET-DET-NOUN'},
 {'current': 'วอน',
  'current_pos': 'NOUN',
  'next': 'ซาน',
  'next_pos': 'NOUN',
  'prev': 'ทั้งเมือง',
  'prev_pos': 'DET',
  'conj-word': 'อีก-ทั้งเมือง-วอน-ซาน- ',
  'conj-pos': 'DET-NOUN-NOUN'},
 {'current': 'ซาน',
  'current_pos': 'NOUN',
  'next': ' ',
  'next_pos': 'PUNCT',
  'prev': 'วอน',
  'prev_pos': 'NOUN',
  'conj-word': 'ทั้งเมือง-วอน-ซาน- -เป็น',
  'conj-pos': 'NOUN-NOUN-PUNCT'},
 {'current': ' ',
  'current_pos': 'PUNCT',
  'next': 'เป็น',
  'next_pos': 'VERB',
  'prev': 'ซาน',
  'prev_pos': 'NOUN',
  'conj-word': 'วอน-ซาน- -เป็น-เมือง',
  'conj-pos': 'NOUN-PUNCT-VERB'},
 {'current': 'เป็น',
  'current_pos': 'VERB',
  'n

In [ ]:
feature_seq_list[-1]

In [ ]:
dev_list = [[(x[0], x[1]) for x in pos_tag(s, corpus='orchid_ud')] for s in dev_seq_list]

In [ ]:
test_list = [[(x[0], x[1]) for x in pos_tag(s, corpus='orchid_ud')] for s in test_seq_list]

In [ ]:
feature_functions = [current, current_pos, next, next_pos, prev, prev_pos, conjunctive_word, conjunctive_pos]
feature_seq_list = [featurize(feature_functions, token_list) for token_list in dev_list]
feature_seq_list[0]

In [ ]:
feature_functions = [current, current_pos, next, next_pos, prev, prev_pos, conjunctive_word, conjunctive_pos]
test_feature_seq_list = [featurize(feature_functions, token_list) for token_list in test_list]
test_feature_seq_list[0]

[{'current': 'สั่ง',
  'current_pos': 'VERB',
  'next': 'คุม',
  'next_pos': 'NOUN',
  'prev': 'START-WORD',
  'prev_pos': 'START-WORD'},
 {'current': 'คุม',
  'current_pos': 'NOUN',
  'next': 'เข้ม',
  'next_pos': 'ADJ',
  'prev': 'START-WORD',
  'prev_pos': 'START-WORD',
  'conj-word': 'สั่ง-คุม-เข้ม-"',
  'conj-pos': 'VERB-NOUN-ADJ'},
 {'current': 'เข้ม',
  'current_pos': 'ADJ',
  'next': '"',
  'next_pos': 'PUNCT',
  'prev': 'คุม',
  'prev_pos': 'NOUN',
  'conj-word': 'สั่ง-คุม-เข้ม-"-ชุมชน',
  'conj-pos': 'NOUN-ADJ-PUNCT'},
 {'current': '"',
  'current_pos': 'PUNCT',
  'next': 'ชุมชน',
  'next_pos': 'NOUN',
  'prev': 'เข้ม',
  'prev_pos': 'ADJ',
  'conj-word': 'คุม-เข้ม-"-ชุมชน-แพง',
  'conj-pos': 'ADJ-PUNCT-NOUN'},
 {'current': 'ชุมชน',
  'current_pos': 'NOUN',
  'next': 'แพง',
  'next_pos': 'ADJ',
  'prev': '"',
  'prev_pos': 'PUNCT',
  'conj-word': 'เข้ม-"-ชุมชน-แพง-เพียบ',
  'conj-pos': 'PUNCT-NOUN-ADJ'},
 {'current': 'แพง',
  'current_pos': 'ADJ',
  'next': 'เพียบ',
  'next_p

In [ ]:
for x_seq, y_seq in zip(token_seq_list, label_seq_list):
  a = len(x_seq) - len(y_seq)
  if a != 0:
    print(len(x_seq) - len(y_seq))

In [ ]:
len(feature_seq_list)

68930

#Train CRF

In [ ]:
import sklearn
sklearn.__version__

'1.2.2'

In [ ]:
import sklearn_crfsuite
crf = sklearn_crfsuite.CRF()
crf.keep_tempfiles = False
crf.model_filename= None
crf.fit(feature_seq_list, label_seq_list)

CRF()

# Evaluate


In [ ]:
pred = crf.predict(feature_seq_list)
pred[0:2]

[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['B_PER',
  'I_PER',
  'I_PER',
  'I_PER',
  'O',
  'O',
  'O',
  'O',
  'B_NUM',
  'O',
  'B_MEA',
  'I_MEA',
  'O',
  'B_NUM',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B_MEA',
  'I_MEA',
  'I_MEA',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B_MEA',
  'O',
  'O',
  'O',
  'O',
  'O']]

In [ ]:
test_pred = crf.predict(test_feature_seq_list)
test_pred[-2:]

[['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'O',
  'B_ORG',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B_ORG',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B_ORG',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O']]

In [ ]:
dev_pred = crf.predict(feature_seq_list)
dev_pred[-2:]

In [ ]:
from sklearn.metrics import classification_report
flat_gold_list = [label for seq in label_seq_list for label in seq]
flat_pred_list = [label for seq in pred for label in seq]
print(classification_report(flat_gold_list, flat_pred_list))

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

       B_BRN       0.86      0.33      0.48       460
       B_DES       0.89      0.84      0.86     16915
       B_DTM       0.87      0.82      0.84     14882
       B_LOC       0.82      0.77      0.79     31202
       B_MEA       0.80      0.76      0.78     32214
      B_NAME       0.00      0.00      0.00        14
       B_NUM       0.74      0.68      0.71     13569
       B_ORG       0.84      0.76      0.80     46454
       B_PER       0.92      0.89      0.91     45666
       B_TRM       0.82      0.60      0.69       898
       B_TTL       0.93      0.95      0.94     21444
       I_BRN       0.88      0.50      0.63       505
       I_DES       0.87      0.83      0.85      6370
       I_DTM       0.88      0.95      0.91     46211
       I_LOC       0.83      0.85      0.84     29985
       I_MEA       0.71      0.89      0.79     25103
      I_NAME       0.00      0.00      0.00        23
       I_NUM       0.82    

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
dev_mapped_list = [list(zip(words, labels)) for words, labels in zip(dev_seq_list, dev_pred)]

In [ ]:
test_mapped_list = [list(zip(words, labels)) for words, labels in zip(test_seq_list, test_pred)]

In [ ]:
len(test_mapped_list)

5250

In [ ]:
from lst20utils import extract_sentences_from_files, extract_entities, retag

In [ ]:
import json
def write_entity_file(sentences, file_name):
    solution = []
    for s in sentences:
        _, entities = extract_entities(s)
        solution.append(entities)
    with open(file_name, mode='w', encoding='utf8') as f:
        json.dump(solution, f, indent=4)
    return solution

In [ ]:
sentences = [[('"', 'O'), ('อ.', 'B_LOC'), ('สม', 'O'), ('พจน์', 'O'), ('"', 'O'), ('แต่ง', 'O'), ('"', 'O'), ('แพนด้า', 'O'), ('น่ารัก', 'O'), ('"', 'O'), (' ', 'O'), ('ตุน', 'O'), ('เพลง', 'O'), ('รอ', 'O'), ('"', 'O'), ('ลูกกรุง', 'O'), ('"', 'O'), ('กลับมา', 'O'), ('อีก', 'O')], [('หลังจากที่', 'O'), ('มี', 'O'), ('ข่าว', 'O'), ('ว่า', 'O'), ('ได้', 'O'), ('แต่งเพลง', 'O'), ('แพนด้า', 'O'), ('น่ารัก', 'O'), (' ', 'O'), ('เพื่อ', 'O'), ('มอบให้', 'O'), ('ลูก', 'O'), ('ของ', 'O'), ('ช่วง', 'O'), ('ช่วง', 'O'), ('และ', 'O'), ('หลินฮุ่ย', 'O'), (' ', 'O'), ('แม้', 'O'), ('จะ', 'O'), ('ยัง', 'O'), ('ไม่', 'O'), ('ทราบ', 'O'), ('ว่า', 'O'), ('ลูก', 'O'), ('หมี', 'O'), ('แพนด้า', 'O'), ('น้อย', 'O'), ('ชื่อ', 'O'), ('อะไรก็ตาม', 'O'), (' ', 'O'), ('อ.', 'O'), ('สม', 'B_PER'), ('พจน์', 'I_PER'), (' ', 'I_PER'), ('สิงห์', 'I_PER'), ('สุวรรณ', 'I_PER'), (' ', 'I_PER'), ('ได้', 'O'), ('ให้', 'O'), ('สัมภาษณ์', 'O'), ('ที่', 'O'), ('บ้าน', 'O'), ('ฝัน', 'O'), ('ตะวัน', 'O'), ('รัญจวน', 'O'), (' ', 'O'), ('ลาดพร้าว', 'O'), (' ', 'O'), ('124', 'O'), (' ', 'O'), ('ว่า', 'O')], [('"', 'O'), ('สาเหตุ', 'O'), ('ที่', 'O'), ('ผม', 'O'), ('แต่งเพลง', 'O'), ('แพนด้า', 'O'), ('น่ารัก', 'O'), (' ', 'O'), ('ก็', 'O'), ('เพราะ', 'O'), ('อยาก', 'O'), ('จะ', 'O'), ('มีส่วน', 'O'), ('ร่วมใน', 'O'), ('การ', 'O'), ('ต้อนรับ', 'O'), ('ครอบครัว', 'O'), ('หมี', 'O'), ('แพนด้า', 'O'), (' ', 'O'), ('ซึ่ง', 'O'), ('ทางการ', 'O'), ('จีน', 'B_LOC'), ('ส่ง', 'O'), ('มา', 'O'), ('อยู่', 'O'), ('ที่', 'O'), ('ประเทศ', 'B_LOC'), ('ไทย', 'I_LOC'), ('ทูต', 'O'), ('วัฒนธรรม', 'O'), (' ', 'O'), ('ผม', 'O'), ('แต่งเพลง', 'O'), ('นี้', 'O'), ('เสร็จ', 'O'), ('เรียบร้อย', 'O'), ('แล้ว', 'O'), (' ', 'O'), ('ถึง', 'O'), ('ได้', 'O'), ('พบ', 'O'), ('กับ', 'O'), ('น้อง', 'O'), ('ออมสิน', 'O'), ('ที่', 'O'), ('บ้าน', 'O'), ('ฝัน', 'O'), ('ฯ', 'O'), (' ', 'O'), ('เมื่อ', 'O'), ('ทาบทาม', 'O'), ('ให้', 'O'), ('ร้องเพลง', 'O'), ('นี้', 'O'), ('น้อง', 'O'), ('ออมสิน', 'O'), ('ก็', 'O'), ('รีบ', 'O'), ('ตกลง', 'O'), ('ทันที', 'O'), (' ', 'O'), ('อีก', 'O'), ('สอง', 'B_MEA'), ('วัน', 'I_MEA'), ('ต่อมา', 'O'), ('ก็', 'O'), ('เข้า', 'O'), ('ห้อง', 'O'), ('อัด', 'O'), ('เรียบร้อย', 'O'), (' ', 'O'), ('ประมาณ', 'O'), ('สัปดาห์', 'O'), ('หน้า', 'O'), ('จะ', 'O'), ('ฟังเพลง', 'O'), ('แพนด้า', 'O'), ('น่ารัก', 'O'), ('ได้ที่', 'O'), (' ', 'O'), ('www', 'O'), ('.', 'B_ORG'), ('you', 'I_ORG'), ('2', 'I_ORG'), ('play', 'I_ORG'), ('.', 'I_ORG'), ('com', 'O')], [('นอกจาก', 'O'), ('เพลง', 'O'), ('แพนด้า', 'O'), ('น่ารัก', 'O'), ('แล้ว', 'O'), (' ', 'O'), ('ผม', 'O'), ('ยัง', 'O'), ('แต่ง', 'O'), ('อีก', 'O'), ('หลาย', 'O'), ('เพลง', 'O'), ('เป็น', 'O'), ('เพลง', 'O'), ('ไม่เชิง', 'O'), ('ลูกกรุง', 'O'), (' ', 'O'), ('เป็น', 'O'), ('เพลง', 'O'), ('สมัยนิยม', 'O'), ('แนว', 'O'), ('ป็อป', 'O'), ('แจ๊ส', 'O'), (' ', 'O'), ('ให้', 'O'), ('คุณ', 'O'), ('ชูใจ', 'O'), (' ', 'O'), ('เพ', 'O'), ('ชรา', 'O'), (' ', 'O'), ('ลูกศิษย์', 'O'), ('คน', 'O'), ('ใหม่', 'O'), ('ร้อง', 'O'), (' ', 'O'), ('และ', 'O'), ('ยัง', 'O'), ('แต่งเพลง', 'O'), ('ใหม่', 'O'), ('ให้', 'O'), ('คุณ', 'O'), ('โฉมฉาย', 'O'), (' ', 'O'), ('อรุณ', 'O'), ('ฉาน', 'O'), (' ', 'O'), ('ร้อง', 'O'), ('อีกด้วย', 'O')], [('ขณะนี้', 'O'), ('กำลัง', 'O'), ('แต่งเพลง', 'O'), ('หนุ่ม', 'O'), ('กางเกงแพร', 'O'), (' ', 'O'), ('สำหรับ', 'O'), ('งาน', 'O'), ('กางเกงแพร', 'O'), ('อิน', 'O'), ('คอนเสิร์ต', 'O'), (' ', 'O'), ('ผม', 'O'), ('คิด', 'O'), ('ว่า', 'O'), ('ปีหน้า', 'O'), ('เพลง', 'O'), ('ลูกกรุง', 'O'), ('คงจะ', 'O'), ('กลับมา', 'O'), ('แน่นอน', 'O'), (' ', 'O'), ('ผม', 'O'), ('จึง', 'O'), ('ได้', 'O'), ('รีบ', 'O'), ('แต่งเพลง', 'O'), ('ไว้', 'O'), ('รอ', 'O'), ('รับ', 'O'), ('การ', 'O'), ('กลับมา', 'O'), ('"', 'O'), (' ', 'O'), ('อ.', 'O'), ('สม', 'O'), ('พจน์', 'O'), ('กล่าว', 'O')], [('พท.', 'O'), ('พร้อม', 'O'), ('หนุน', 'O'), ('จัด', 'O'), ('ประชุม', 'O'), ('สมัชชา', 'O'), ('สหภาพ', 'O'), ('รัฐสภา', 'O'), (' ', 'O'), ('ครั้ง', 'O'), ('ที่', 'O'), (' ', 'O'), ('122', 'O'), (' ', 'O'), ('ที่', 'O'), ('ไทย', 'O'), ('เป็นเจ้าภาพ', 'O')], [('รอง', 'O'), ('หัวหน้าพรรค', 'O'), ('เพื่อ', 'O'), ('ไทย', 'O'), (' ', 'O'), ('ระบุ', 'O'), (' ', 'O'), ('พรรค', 'O'), ('พร้อม', 'O'), ('ให้การ', 'O'), ('สนับสนุน', 'O'), ('การ', 'O'), ('จัด', 'O'), ('ประชุม', 'O'), ('สมัชชา', 'O'), ('สหภาพ', 'O'), ('รัฐสภา', 'O'), (' ', 'O'), ('ครั้ง', 'O'), ('ที่', 'O'), (' ', 'O'), ('122', 'O'), (' ', 'O'), ('ที่', 'O'), ('ประเทศ', 'O'), ('ไทย', 'B_LOC'), ('เป็นเจ้าภาพ', 'O'), ('อย่าง', 'O'), ('เต็มที่', 'O'), (' ', 'O'), ('ภายหลัง', 'O'), ('การหารือ', 'O'), ('กับ', 'O'), ('เลขาธิการ', 'O'), ('สหภาพ', 'O'), ('รัฐสภา', 'O')], [('นาย', 'B_TTL'), ('ปาน', 'B_PER'), ('ป', 'I_PER'), ('รี', 'I_PER'), ('ย์', 'I_PER'), (' ', 'I_PER'), ('พหิทธา', 'I_PER'), ('นุ', 'I_PER'), ('กร', 'I_PER'), (' ', 'I_PER'), ('รอง', 'O'), ('หัวหน้าพรรค', 'O'), ('เพื่อ', 'O'), ('ไทย', 'O'), (' ', 'O'), ('เปิดเผย', 'O'), ('ว่า', 'O'), (' ', 'O'), ('เมื่อ', 'O'), ('วันที่', 'B_DTM'), (' ', 'I_DTM'), ('11', 'I_DTM'), (' ', 'I_DTM'), ('สิงหาคม', 'I_DTM'), (' ', 'O'), ('ที่ผ่านมา', 'O'), (' ', 'O'), ('ได้', 'O'), ('มีโอกาส', 'O'), ('พบ', 'O'), ('กับ', 'O'), (' ', 'O'), ('นาย', 'B_TTL'), ('อันเดอร์', 'B_PER'), ('ส', 'I_PER'), (' ', 'I_PER'), ('บี', 'I_PER'), ('.', 'I_PER'), (' ', 'I_PER'), ('จอห์น', 'I_PER'), ('สัน', 'I_PER'), (' ', 'I_PER'), ('(', 'O'), ('Mr', 'O'), ('.', 'O'), (' ', 'O'), ('Anders', 'O'), (' ', 'O'), ('B', 'O'), ('.', 'O'), (' ', 'O'), ('Johnsson', 'O'), (')', 'O'), (' ', 'O'), ('เลขาธิการ', 'O'), ('สหภาพ', 'O'), ('รัฐสภา', 'O'), (' ', 'O'), ('หรือ', 'O'), (' ', 'O'), ('IPU', 'O'), (' ', 'O'), ('(', 'O'), ('Inter-Parliamentary', 'O'), (' ', 'O'), ('Union', 'O'), (')', 'O'), (' ', 'O'), ('และ', 'O'), (' ', 'O'), ('นาย', 'B_TTL'), ('ต่อ', 'B_PER'), ('พงษ์', 'I_PER'), (' ', 'I_PER'), ('ไชย', 'I_PER'), ('สาส์น', 'I_PER'), (' ', 'I_PER'), ('ประธาน', 'O'), ('คณะกรรมาธิการ', 'O'), ('การต่างประเทศ', 'O'), (' ', 'O'), ('สภาผู้แทนราษฎร', 'O'), (' ', 'O'), ('ซึ่ง', 'O'), (' ', 'O'), ('นาย', 'B_TTL'), ('อันเดอร์', 'B_PER'), ('ส', 'I_PER'), (' ', 'I_PER'), ('ได้', 'O'), ('เดินทาง', 'O'), ('มา', 'O'), ('เยือน', 'O'), ('ประเทศ', 'O'), ('ไทย', 'B_LOC'), ('เพื่อ', 'O'), ('ต้องการ', 'O'), ('พบปะ', 'O'), ('และ', 'O'), ('แลกเปลี่ยนความคิดเห็น', 'O'), ('กับ', 'O'), ('รัฐบาล', 'O'), ('และ', 'O'), ('หน่วยงาน', 'O'), ('ต่างๆ', 'O'), (' ', 'O'), ('ของ', 'O'), ('ไทย', 'B_LOC')], [('เพื่อ', 'O'), ('เตรียม', 'O'), ('ความพร้อม', 'O'), ('สำหรับ', 'O'), ('จัด', 'O'), ('การประชุม', 'O'), ('สมัชชา', 'O'), ('สหภาพ', 'O'), ('รัฐสภา', 'O'), (' ', 'O'), ('(', 'O'), ('IPU', 'O'), (' ', 'O'), ('Assembly', 'O'), (')', 'O'), (' ', 'O'), ('ครั้ง', 'O'), ('ที่', 'O'), (' ', 'O'), ('122', 'O'), (' ', 'O'), ('ระหว่าง', 'O'), ('เดือน', 'B_DTM'), ('มีนาคม', 'I_DTM'), (' ', 'I_DTM'), ('-', 'I_DTM'), (' ', 'I_DTM'), ('เมษายน', 'I_DTM'), (' ', 'I_DTM'), ('ปี', 'I_DTM'), (' ', 'I_DTM'), ('2553', 'I_DTM'), (' ', 'O'), ('ที่', 'O'), ('ประเทศ', 'O'), ('ไทย', 'B_LOC'), (' ', 'O'), ('โดย', 'O'), ('ได้', 'O'), ('มี', 'O'), ('การ', 'O'), ('แลกเปลี่ยนความคิดเห็น', 'O'), (' ', 'O'), ('และ', 'O'), ('ขอ', 'O'), ('การ', 'O'), ('สนับสนุน', 'O'), ('จาก', 'O'), ('พรรค', 'O'), ('เพื่อ', 'O'), ('ไทย', 'O'), ('ใน', 'O'), ('ฐานะ', 'O'), ('ฝ่ายค้าน', 'O')], [('สำหรับ', 'O'), ('การ', 'O'), ('จัดงาน', 'O'), ('ใน', 'O'), ('ปีหน้า', 'O'), ('ด้วย', 'O'), (' ', 'O'), ('รอง', 'O'), ('หัวหน้าพรรค', 'O'), ('เพื่อ', 'O'), ('ไทย', 'O'), (' ', 'O'), ('กล่าวว่า', 'O'), (' ', 'O'), ('พรรค', 'O'), ('เพื่อ', 'O'), ('ไทย', 'O'), ('พร้อม', 'O'), ('สนับสนุน', 'O'), ('การ', 'O'), ('จัดงาน', 'O'), ('ประชุม', 'O'), ('สมัชชา', 'O'), ('สหภาพ', 'O'), ('รัฐสภา', 'O'), (' ', 'O'), ('และ', 'O'), ('จะ', 'O'), ('ช่วย', 'O'), ('ผลักดัน', 'O'), ('ให้', 'O'), ('งบประมาณ', 'O'), ('ที่', 'O'), ('ใช้', 'O'), ('สำหรับ', 'O'), ('การ', 'O'), ('จัดงาน', 'O'), (' ', 'O'), ('ซึ่ง', 'O'), ('อยู่', 'O'), ('ระหว่าง', 'O'), ('การพิจารณา', 'O'), ('ของ', 'O'), ('คณะกรรมาธิการ', 'O'), ('พิจารณา', 'O'), ('งบประมาณ', 'O'), ('ฯ', 'O'), (' ', 'O'), ('สภาผู้แทนราษฎร', 'B_ORG'), (' ', 'O'), ('ให้', 'O'), ('ผ่าน', 'O'), ('ความ', 'O'), ('เห็นชอบ', 'O'), ('จาก', 'O'), ('สภา', 'B_ORG'), ('โดยเร็ว', 'O')]]

In [ ]:
pred4_dev_entities = write_entity_file(dev_mapped_list, 'pred4_dev_entities.json')

In [ ]:
predicted5_test_entities = write_entity_file(test_mapped_list, 'predicted5_test_entities.json')

In [ ]:
!wc -l predicted1_test_entities.json

41512 predicted1_test_entities.json


In [ ]:
predicted5_test_entities[:10]

[[],
 [('ORG', 'สำนักงบฯ'), ('NUM', '5.3 พันล้าน'), ('MEA', '50')],
 [('TTL', 'นาย'),
  ('PER', 'อรรถวิชช์ สุวรรณภักดี'),
  ('DES', 'ส.ส.'),
  ('ORG', 'กทม.'),
  ('ORG', 'พรรคประชาธิปัตย์')],
 [],
 [],
 [('NUM', '2.'), ('NUM', '170'), ('MEA', '50'), ('ORG', 'สพช.')],
 [('NUM', '3.'), ('MEA', '20')],
 [('NUM', '4.')],
 [('ORG', 'กองปราบปราม'), ('DTM', 'วันที่ 4 ส.ค.')],
 []]

In [ ]:
import json
# load the JSON file
with open('/content/dev_entities.json') as f:
    data = json.load(f)

# count the number of sentences
num_sentences = len(data)

print('Number of sentences:', num_sentences)

Number of sentences: 5620


In [ ]:
!python3 evaluate.py dev_entities.json pred4_dev_entities.json

{
  "0 Overall F1": 0.6998728509038643,
  "0 Overall P": 0.8325660923319742,
  "0 Overall R": 0.6036620255578867,
  "1 ORG F1": 0.7337579617834394,
  "2 PER F1": 0.8337769890564922,
  "3 MEA F1": 0.5306806282722514,
  "4 LOC F1": 0.6540585774058578,
  "5 TTL F1": 0.8790712673331184,
  "6 DTM F1": 0.6745298055467007,
  "7 NUM F1": 0.5281346920691183,
  "8 DES F1": 0.681383370125092,
  "9 MISC F1": 0,
  "10 TRM F1": 0.5572519083969466,
  "11 BRN F1": 0.4285714285714285
}
